In [1]:
import pandas as pd
import os
import pickle as pkl
from pprint import pprint
from plotly import express as px
from lightgbm import LGBMRegressor

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.model.utils import qwk_obj

In [4]:
df = pd.DataFrame()
names = None

for fold in range(config.lgbm_n_folds):
    model: LGBMRegressor = pkl.load(open(f"output/LGBM/{fold}.pkl", "rb"))["model"]
    importances = model.feature_importances_

    if not names:
        names = model.feature_name_
        df = pd.concat([df, pd.DataFrame({"name": names})], axis=1)

    temp = pd.DataFrame({f"LGBM_{fold}": importances})
    df = pd.concat([df, temp], axis=1)

In [5]:
df.head(10)

,name,LGBM_0,LGBM_1,LGBM_2,LGBM_3,LGBM_4,LGBM_5,LGBM_6,LGBM_7,LGBM_8,...,LGBM_15,LGBM_16,LGBM_17,LGBM_18,LGBM_19,LGBM_20,LGBM_21,LGBM_22,LGBM_23,LGBM_24
0,deberta_m0_c0,118,131,154,101,101,91,127,122,62,...,77,65,141,140,123,129,109,136,146,91
1,deberta_m0_c1,98,102,99,81,91,90,99,100,41,...,79,53,110,113,95,84,94,112,119,78
2,deberta_m0_c2,105,107,110,93,95,85,110,110,67,...,85,65,114,100,94,97,108,105,105,86
3,deberta_m0_c3,53,80,88,67,73,55,63,73,50,...,49,41,71,67,71,53,58,78,93,54
4,deberta_m0_c4,73,84,86,67,60,64,69,70,39,...,48,42,75,71,67,81,71,76,93,63
5,deberta_m0_c5,33,52,69,33,24,36,33,55,17,...,24,26,41,58,43,39,38,52,60,36
6,deberta_m1_c0,47,62,94,56,47,38,59,75,27,...,33,32,70,66,68,62,48,85,92,45
7,deberta_m1_c1,71,70,78,73,62,66,74,73,47,...,43,40,69,61,46,80,69,58,71,62
8,deberta_m1_c2,112,114,120,107,96,100,103,101,61,...,84,82,108,121,118,107,117,116,113,97
9,deberta_m1_c3,53,52,68,37,45,43,57,50,33,...,42,30,51,59,49,53,42,59,76,36


In [6]:
df["importance"] = df.loc[:, "LGBM_0":].median(axis=1)
df.sort_values(by="importance", ascending=False, inplace=True)
df = df[["name", "importance"]]

## Overall

Top 10 most important features

In [7]:
df.head(10)

,name,importance
18,deberta_m3_c0,119.0
0,deberta_m0_c0,114.0
12,deberta_m2_c0,109.0
8,deberta_m1_c2,107.0
20,deberta_m3_c2,105.0
2,deberta_m0_c2,97.0
1,deberta_m0_c1,94.0
14,deberta_m2_c2,88.0
36,deberta_m6_c0,77.0
38,deberta_m6_c2,77.0


Top 10 least important features

In [8]:
df.tail(10)

,name,importance
164,tfidf_55,0.0
166,tfidf_57,0.0
175,tfidf_66,0.0
168,tfidf_59,0.0
169,tfidf_60,0.0
170,tfidf_61,0.0
171,tfidf_62,0.0
172,tfidf_63,0.0
174,tfidf_65,0.0
177,tfidf_68,0.0


## Broad Groups

In [9]:
def get_broad_group(name):
    group = name.split("_")[0]

    if group == "tfidf" and name.split("_")[1] == "count":
        return "tfidf_count"

    return group

In [10]:
df["broad_group"] = df["name"].map(get_broad_group)

In [11]:
px.box(
    df,
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

From an high level, it looks like
* `score` probabilities from DeBERTa model has he highest importance.
* `Paragraph` and `sentence` features also seem to be quite important as well.
* `word` and `tfidf_count` seems to be the least important.

In [12]:
def get_count_type(name):
    return "_".join(name.split("_")[1:3])

In [13]:
df["count_type"] = df["name"].map(get_count_type)

## Broad Operations

In [14]:
def get_operation_type(name):
    last_word = name.split("_")[-1]
    if last_word in [
        "sum",
        "max",
        "mean",
        "min",
        "q1",
        "q3",
    ]:
        return last_word

    if name.find("len_goe") != -1:
        return "greater_than"
    elif name.find("len_l") != -1:
        return "less_than"
    return "None"

In [15]:
df["operation_type"] = df["name"].map(get_operation_type)

In [16]:
px.box(
    df,
    x="importance",
    color="operation_type",
    range_x=(df.importance.min(), df.importance.max()),
)

Broadly, `min`, `kurtosis`, `q1`, `first` are worthless, regardless of broad group.

### Greater Than

In [17]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

In [18]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [19]:
df[(df.operation_type == "greater_than") & (df.importance >= 10)]

,name,importance,broad_group,count_type,operation_type
69,paragraph_sentence_count_len_goe_16,32.0,paragraph,sentence_count,greater_than
77,paragraph_error_count_len_goe_16,12.0,paragraph,error_count,greater_than


**Verdict**

1. For paragraphs
   1. Count only sentence and error. Upper limit be 15.
2. For sentences
   1. Count only error and word. Upper limit be 20.

### Less Than

In [20]:
px.box(
    df[df.operation_type == "less_than"],
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

In [21]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [22]:
df[(df.operation_type == "less_than") & (df.importance >= 10)]

,name,importance,broad_group,count_type,operation_type
68,paragraph_sentence_count_len_l_16,43.0,paragraph,sentence_count,less_than
76,paragraph_error_count_len_l_16,21.0,paragraph,error_count,less_than
99,sentence_word_count_len_l_16,15.0,sentence,word_count,less_than


**Verdict**

1. For paragraphs: Count only sentence and error. Upper limit be 20.
2. For sentences: Count only error and word. Upper limit be 20.

## Paragraph Features

In [23]:
paragraph_features = df.loc[df.broad_group == "paragraph"]
print("Shape of paragraph features:", paragraph_features.shape)

Shape of paragraph features: (36, 5)


In [24]:
px.box(
    paragraph_features,
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [25]:
paragraph_features[
    (paragraph_features.count_type == "char_count")
    & (paragraph_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
47,paragraph_char_count_sum,48.0,paragraph,char_count,sum
46,paragraph_char_count_max,32.0,paragraph,char_count,max
45,paragraph_char_count_mean,25.0,paragraph,char_count,mean
57,paragraph_char_count_q3,21.0,paragraph,char_count,q3
56,paragraph_char_count_q1,7.0,paragraph,char_count,q1


In [26]:
paragraph_features[
    (paragraph_features.count_type == "word_count")
    & (paragraph_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
50,paragraph_word_count_sum,71.0,paragraph,word_count,sum
49,paragraph_word_count_max,27.0,paragraph,word_count,max
48,paragraph_word_count_mean,25.0,paragraph,word_count,mean
59,paragraph_word_count_q3,19.0,paragraph,word_count,q3
58,paragraph_word_count_q1,9.0,paragraph,word_count,q1


## TFIDF and Count

In [27]:
tfidf_features = df.loc[df.broad_group == "tfidf"]
print("Shape of tfidf features:", tfidf_features.shape)

Shape of tfidf features: (100, 5)


In [28]:
px.box(
    tfidf_features,
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

TFIDF count features are worthless.

In [29]:
tfidf_features[
    (tfidf_features.broad_group == "tfidf") & (tfidf_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
135,tfidf_26,37.0,tfidf,26,None
109,tfidf_0,31.0,tfidf,0,None
110,tfidf_1,25.0,tfidf,1,None
116,tfidf_7,7.0,tfidf,7,None
176,tfidf_67,5.0,tfidf,67,None
117,tfidf_8,3.0,tfidf,8,None
149,tfidf_40,3.0,tfidf,40,None
165,tfidf_56,3.0,tfidf,56,None
150,tfidf_41,2.0,tfidf,41,None
156,tfidf_47,2.0,tfidf,47,None


## DeBERTa Features

In [30]:
deberta_features = df.loc[df.broad_group.isin(["deberta"])]
print("Shape of deberta features:", deberta_features.shape)

Shape of deberta features: (42, 5)


In [31]:
def get_model_num(name):
    return int(name.split("_")[1][1])


deberta_features["model"] = deberta_features.name.map(get_model_num)

/tmp/ipykernel_16968/962180890.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
px.box(
    deberta_features,
    x="importance",
    color="model",
    range_x=(df.importance.min(), df.importance.max()),
)

## Word Features

In [33]:
word_features = df.loc[df.broad_group.isin(["word"])]
print("Shape of word features:", word_features.shape)

Shape of word features: (60, 5)


In [34]:
def get_specific(name):
    return name.split("_")[1]

word_features["specific"] = word_features["name"].map(get_specific)

/tmp/ipykernel_16968/957994367.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
px.box(
    word_features,
    x="importance",
    color="specific",
    # range_x=(df.importance.min(), df.importance.max()),
)